# Hand written digits recognition with Convolutional Neural Network (Keras)

In this notebook we are going to build a Convolutional Neural Network model to recognize hand written digits using Keras.

## Importing packages

In [13]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, Dropout, MaxPooling2D
from keras.models import Model
from keras.utils import np_utils, plot_model
import matplotlib.pyplot as plt

%matplotlib inline

## Loading data

In [8]:
from keras.datasets import mnist
(X_train_orig, y_train_orig), (X_test_orig, y_test_orig) = mnist.load_data()

Let's have a look at the shape of the dataset

In [9]:
print('X_train_orig shape: '+str(X_train_orig.shape))
print('y_train_orig shape: '+str(y_train_orig.shape))
print('X_test_orig shape: '+str(X_test_orig.shape))
print('y_test_orig shape: '+str(y_test_orig.shape))

X_train_orig shape: (60000, 28, 28)
y_train_orig shape: (60000,)
X_test_orig shape: (10000, 28, 28)
y_test_orig shape: (10000,)


## Preprocessing data

The desired shape for our input layer is (m, 28, 28, 1).

In [11]:
X_train = X_train_orig.reshape(-1,28,28,1)
X_test = X_test_orig.reshape(-1,28,28,1)
print('X_train shape: '+str(X_train.shape))
print('X_test shape: '+str(X_test.shape))

X_train shape: (60000, 28, 28, 1)
X_test shape: (10000, 28, 28, 1)


Let's normalize data

In [12]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

Now we have to Convert 1-dimensional class arrays to 10-dimensional class matrices: **one hot encoding**

In [15]:
y_train = np_utils.to_categorical(y_train_orig, 10)
y_test = np_utils.to_categorical(y_test_orig, 10)
print('y_train shape: '+str(y_train.shape))
print('y_test shape: '+str(y_test.shape))

y_train shape: (60000, 10)
y_test shape: (10000, 10)


## Building the model

We'll build a model to classify the images in the MNIST dataset using the following CNN architecture:

1. Convolutional Layer #1: Applies 32 5x5 filters (extracting 5x5-pixel subregions) with "same" padding and applying Batch Normalization and ReLU activation function
2. Pooling Layer #1: Performs max pooling with a 2x2 filter and stride of 2 (which specifies that pooled regions do not overlap)
3. Convolutional Layer #2: Applies 64 5x5 filters with "same" padding and applying Batch Normalization and ReLU activation function
4. Pooling Layer #2: Again, performs max pooling with a 2x2 filter and stride of 2
5. Dense Layer #1: 128 neurons, with dropout regularization rate of 0.5 (probability of 0.4 that any given element will be dropped during training)
6. Dense Layer #2 (Logits Layer): 10 neurons, one for each digit target class (0–9).

### Structure of the model

In [19]:
def DigitRecognition(input_shape):
    X_input = Input(input_shape)
    
    X = Conv2D(32, (5,5), strides=(1,1), name='Conv0')(X_input)
    X = BatchNormalization(axis=3, name='bn0')(X)
    X = Activation('relu')(X)
    
    X = MaxPooling2D((2,2), name='max_pool0')(X)
    
    X = Conv2D(64, (5,5), strides=(1,1), name='Conv1')(X)
    X = BatchNormalization(axis=3, name='bn1')(X)
    X = Activation('relu')(X)
    
    X = MaxPooling2D((2,2), name='max_pool1') (X)
    
    X = Flatten()(X)
    X = Dense(128, activation='relu', name='FC0')(X)
    X = Dropout(0.5)(X)
    X = Dense(10, activation='softmax', name='FC1')(X)
    
    model = Model(inputs=X_input, outputs=X, name='DigitRecognitionModel')
    
    return model
    

### Creating model

In [20]:
DigitRecognitionModel = DigitRecognition((28,28,1))

### Compiling the model

In [21]:
DigitRecognitionModel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### Trainning the model

In [23]:
DigitRecognitionModel.fit(x=X_train, y=y_train, batch_size=32, epochs=10, verbose=1)

Epoch 1/10
60000/60000 [==============================] - 416s 7ms/step - loss: 0.2013 - acc: 0.9396
Epoch 2/10
60000/60000 [==============================] - 404s 7ms/step - loss: 0.0928 - acc: 0.9727
Epoch 3/10
60000/60000 [==============================] - 413s 7ms/step - loss: 0.0729 - acc: 0.9796
Epoch 4/10
60000/60000 [==============================] - 437s 7ms/step - loss: 0.0569 - acc: 0.9836
Epoch 5/10
60000/60000 [==============================] - 443s 7ms/step - loss: 0.0473 - acc: 0.9860
Epoch 6/10
60000/60000 [==============================] - 434s 7ms/step - loss: 0.0412 - acc: 0.9878
Epoch 7/10
60000/60000 [==============================] - 430s 7ms/step - loss: 0.0366 - acc: 0.9891
Epoch 8/10
60000/60000 [==============================] - 451s 8ms/step - loss: 0.0307 - acc: 0.9908
Epoch 9/10
60000/60000 [==============================] - 451s 8ms/step - loss: 0.0279 - acc: 0.9914
Epoch 10/10
60000/60000 [==============================] - 433s 7ms/step - loss: 0.0251 - a

### Evaluating the model

In [25]:
preds = DigitRecognitionModel.evaluate(x=X_test, y=y_test)
print("loss="+str(preds[0]))
print("test accuracy="+str(preds[1]))

10000/10000 [==============================] - 25s 3ms/step
loss=0.022998055769
test accuracy=0.9925
